In [1]:
from correlation import *

import pandas as pd
import numpy as np

In [2]:
genes = pd.read_csv('data/gene_fpkm.txt', sep='\t')

genes = genes.drop_duplicates(subset=['GENE_ID'], keep='first')

genes = genes.rename(columns={'GENE_ID': 'MMRF'})

genes = genes.set_index('MMRF')

del genes['Location']

genes = genes[[i for i in genes.columns if '_1_' in i]]

genes.columns = [i.split('_')[1] for i in genes.columns]

genes = genes.T

genes.index = [int(i) for i in genes.index]

genes.iloc[:10,:10]

MMRF,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167
2438,0.645325,0.000000,32.1035,7.66910,7.06020,0.719675,15.395500,50.19260,11.32710,18.69440
1786,3.519150,0.000000,29.3108,4.96608,1.50599,0.705555,1.283390,22.90920,7.74184,12.50950
1332,6.873060,0.000000,41.2908,8.25159,4.64106,0.927851,0.072551,46.13120,13.67120,18.62340
2562,5.441500,0.000000,27.8386,8.23226,2.71763,0.415857,0.064742,14.38730,15.64070,17.71990
1797,0.000000,0.000000,36.8206,7.72755,3.28094,0.220043,0.000000,17.05950,17.28590,13.67230
1861,0.404031,0.000000,41.0652,5.04330,1.54744,49.960800,1.267370,9.27582,8.13927,15.71420
1823,3.682560,0.000000,60.5635,2.57942,1.71766,0.545700,0.165141,38.00400,9.23173,15.48300
2018,0.795386,0.000000,29.0764,4.87317,2.25814,0.311901,0.685952,14.98340,11.31560,13.93950
2268,2.714760,0.491686,31.3695,2.17873,2.26555,172.304000,1.235570,21.47350,11.27650,10.58360
2570,0.124405,0.000000,39.5649,1.60298,1.19724,0.925557,0.222275,17.35290,5.64112,6.85792


In [3]:
clinical_data = pd.read_csv('data/iss_fish_therapy_response.csv', sep='\t')

clinical_data['MMRF'] = [int(i.replace('MMRF', '')) for i in clinical_data['MMRF']]

clinical_data = clinical_data.loc[clinical_data['Days-to-Progression'].notnull(),:]

clinical_data = clinical_data.set_index('MMRF')

#del clinical_data['Days-to-Overall-Survivel']

clinical_data.iloc[:10, :10]

,Best-Response-FirstLine,Days-to-Overall-Survival,Days-to-Progression,Patient-FirstResponse,ISS,del13q14,del13q34,del17p13,Age,Race
MMRF,,,,,,,,,,
1007,SD,327.0,326.0,NaN,III,NaN,NaN,NaN,78,W
1011,VGPR,1969.0,1612.0,PR,III,NaN,NaN,NaN,50,W
1013,CR,1759.0,1638.0,PR,III,NaN,NaN,NaN,66,W
1016,VGPR,692.0,691.0,VGPR,I,B,B,ND,56,W
1017,VGPR,508.0,507.0,PR,I,NaN,NaN,NaN,75,W
1020,CR,1826.0,713.0,VGPR,I,NaN,NaN,NaN,60,W
1021,VGPR,1932.0,622.0,PR,I,B,B,ND,54,W
1024,PR,1927.0,653.0,PR,II,NaN,NaN,NaN,79,W
1030,CR,1984.0,1983.0,PR,I,ND,ND,ND,65,W


In [4]:
groups = {1: (['SCR'], ['CR', 'VGPR', 'PR', 'SD', 'PD']),
          2: (['SCR', 'CR'], ['VGPR', 'PR', 'SD', 'PD']),
          3: (['SCR', 'CR', 'VGPR'], ['PR', 'SD', 'PD']),
          4: (['SCR', 'CR', 'VGPR', 'PR'], ['SD', 'PD']),
          5: (['SCR', 'CR', 'VGPR', 'PR', 'SD'], ['PD']),
          6: (['SCR'], ['CR', 'VGPR'], ['PR', 'SD', 'PD']),
          7: (['SCR'], ['CR', 'VGPR', 'PR'], ['SD', 'PD'])}

var_resp = ['Days-to-Progression']

clinical_data['Greater18Months-Days-to-Progression'] = [int(i > 18 * 30) for i in clinical_data['Days-to-Progression']]

var_resp.append('Greater18Months-Days-to-Progression')

for i in range(1,6):
    
    var_resp.append('Group{}-From-Best-Response-FirstLine'.format(i))
    
    clinical_data[var_resp[-1]] = clinical_data['Best-Response-FirstLine'].apply(lambda x: 0 if x in groups[i][0] else 1)

clinical_data['Best-Response-FirstLine-ID'] = \
    clinical_data['Best-Response-FirstLine'].map({'SD': 1, 'VGPR': 2, 'CR': 3, 'PR': 3, 'SCR': 4, 'PD': 5, None: 0, np.nan: 0})

var_resp.append('Best-Response-FirstLine-ID')

clinical_data[var_resp].head()

,Days-to-Progression,Greater18Months-Days-to-Progression,Group1-From-Best-Response-FirstLine,Group2-From-Best-Response-FirstLine,Group3-From-Best-Response-FirstLine,Group4-From-Best-Response-FirstLine,Group5-From-Best-Response-FirstLine,Best-Response-FirstLine-ID
MMRF,,,,,,,,
1007,326.0,0,1,1,1,1,0,1
1011,1612.0,1,1,1,0,0,0,2
1013,1638.0,1,1,0,0,0,0,3
1016,691.0,1,1,1,0,0,0,2
1017,507.0,0,1,1,0,0,0,2


In [5]:
clinical_data[var_resp[1:-1]].to_csv('data/output/derived_dependent_variables.csv', index=False, sep=',')

clinical_data[var_resp[1:-1]].head()

,Greater18Months-Days-to-Progression,Group1-From-Best-Response-FirstLine,Group2-From-Best-Response-FirstLine,Group3-From-Best-Response-FirstLine,Group4-From-Best-Response-FirstLine,Group5-From-Best-Response-FirstLine
MMRF,,,,,,
1007,0,1,1,1,1,0
1011,1,1,1,0,0,0
1013,1,1,0,0,0,0
1016,1,1,1,0,0,0
1017,0,1,1,0,0,0


# Gene Expression

In [6]:
all_data = clinical_data[var_resp].join(genes, how='inner')

all_data.iloc[:10, :7]

,Days-to-Progression,Greater18Months-Days-to-Progression,Group1-From-Best-Response-FirstLine,Group2-From-Best-Response-FirstLine,Group3-From-Best-Response-FirstLine,Group4-From-Best-Response-FirstLine,Group5-From-Best-Response-FirstLine
1021,622.0,1,1,1,0,0,0
1024,653.0,1,1,1,1,0,0
1030,1983.0,1,1,0,0,0,0
1031,1328.0,1,1,1,0,0,0
1032,907.0,1,1,1,0,0,0
1033,217.0,0,1,1,1,0,0
1038,1752.0,1,1,1,0,0,0
1045,608.0,1,1,1,0,0,0
1048,1769.0,1,1,1,0,0,0
1068,520.0,0,1,1,1,1,0


In [8]:
pearsons = compute_pearson(all_data.copy(), var_resp)

pearsons.to_csv('data/output/correlation_gene_x_response_vars_pearson.csv', sep=',', index=False)

pearsons.head()

463848


,IND_VAR,DEP_VAR,pearson
0,ENSG00000000003,Days-to-Progression,0.048308
1,ENSG00000000003,Greater18Months-Days-to-Progression,0.073723
2,ENSG00000000003,Group1-From-Best-Response-FirstLine,0.031008
3,ENSG00000000003,Group2-From-Best-Response-FirstLine,0.037248
4,ENSG00000000003,Group3-From-Best-Response-FirstLine,-0.019815


In [9]:
distcorrs = compute_distcorr(all_data.copy(), var_resp)

distcorrs.to_csv('data/output/correlation_gene_x_response_vars_distcorr.csv', sep=',', index=False)

distcorrs.head()

463848


,IND_VAR,DEP_VAR,distcorr
0,ENSG00000000003,Days-to-Progression,0.951692
1,ENSG00000000003,Greater18Months-Days-to-Progression,0.926277
2,ENSG00000000003,Group1-From-Best-Response-FirstLine,0.968992
3,ENSG00000000003,Group2-From-Best-Response-FirstLine,0.962752
4,ENSG00000000003,Group3-From-Best-Response-FirstLine,1.019815


In [10]:
kss = compute_ks(all_data.copy(), var_resp)

kss.to_csv('data/output/correlation_gene_x_response_vars_ks.csv', sep=',', index=False)

kss.head()

463848


,IND_VAR,DEP_VAR,ks
0,ENSG00000000003,Greater18Months-Days-to-Progression,0.083805
1,ENSG00000000003,Group1-From-Best-Response-FirstLine,0.748310
2,ENSG00000000003,Group2-From-Best-Response-FirstLine,0.855860
3,ENSG00000000003,Group3-From-Best-Response-FirstLine,0.855370
4,ENSG00000000003,Group4-From-Best-Response-FirstLine,0.110446


In [11]:
#mics = compute_mic(all_data.copy(), var_resp)

#mics.to_csv('data/output/correlation_gene_x_response_vars_mic.csv', sep=',', index=False)

# Clinical Data

In [12]:
clinical = clinical_data[['ISS', 'Age', 'Race', 'DPRT', 'First-line-Therapy'] + var_resp].copy()

for col in clinical.columns:
    if clinical[col].dtype == 'object':
        clinical[col] = pd.factorize(clinical[col])[0] + 1
        
clinical[clinical.isnull()] = 0

#for c in ['Best-Response-FirstLine', 'Patient-FirstResponse', 'First-line-Therapy-Class', 'First-line-Starting-Treatment-Regimen']:
#    del clinical[c]

clinical.head()

,ISS,Age,Race,DPRT,First-line-Therapy,Days-to-Progression,Greater18Months-Days-to-Progression,Group1-From-Best-Response-FirstLine,Group2-From-Best-Response-FirstLine,Group3-From-Best-Response-FirstLine,Group4-From-Best-Response-FirstLine,Group5-From-Best-Response-FirstLine,Best-Response-FirstLine-ID
MMRF,,,,,,,,,,,,,
1007,1,78,1,1,0,326.0,0,1,1,1,1,0,1
1011,1,50,1,2,1,1612.0,1,1,1,0,0,0,2
1013,1,66,1,2,2,1638.0,1,1,0,0,0,0,3
1016,2,56,1,1,3,691.0,1,1,1,0,0,0,2
1017,2,75,1,1,4,507.0,0,1,1,0,0,0,2


In [13]:
pearsons = compute_pearson(clinical.copy(), var_resp)

pearsons.to_csv('data/output/correlation_clinical_x_response_vars_pearson.csv', sep=',', index=False)

pearsons.head()

40


,IND_VAR,DEP_VAR,pearson
0,ISS,Days-to-Progression,0.099593
1,ISS,Greater18Months-Days-to-Progression,0.143548
2,ISS,Group1-From-Best-Response-FirstLine,-0.028350
3,ISS,Group2-From-Best-Response-FirstLine,-0.011623
4,ISS,Group3-From-Best-Response-FirstLine,-0.121119


In [14]:
distcorrs = compute_distcorr(clinical.copy(), var_resp)

distcorrs.to_csv('data/output/correlation_clinical_x_response_vars_distcorr.csv', sep=',', index=False)

distcorrs.head()

40


,IND_VAR,DEP_VAR,distcorr
0,ISS,Days-to-Progression,0.900407
1,ISS,Greater18Months-Days-to-Progression,0.856452
2,ISS,Group1-From-Best-Response-FirstLine,1.028350
3,ISS,Group2-From-Best-Response-FirstLine,1.011623
4,ISS,Group3-From-Best-Response-FirstLine,1.121119


In [15]:
mics = compute_mic(clinical.copy(), var_resp)

mics.to_csv('data/output/correlation_clinical_x_response_vars_mic.csv', sep=',', index=False)

mics.head()

40


,IND_VAR,DEP_VAR,mic
0,ISS,Days-to-Progression,0.166733
1,ISS,Greater18Months-Days-to-Progression,0.021525
2,ISS,Group1-From-Best-Response-FirstLine,0.006579
3,ISS,Group2-From-Best-Response-FirstLine,0.004402
4,ISS,Group3-From-Best-Response-FirstLine,0.018204


In [16]:
kss = compute_ks(clinical.copy(), var_resp)

kss.to_csv('data/output/correlation_clinical_x_response_vars_ks.csv', sep=',', index=False)

kss.head()

40


,IND_VAR,DEP_VAR,ks
0,ISS,Greater18Months-Days-to-Progression,0.002051
1,ISS,Group1-From-Best-Response-FirstLine,0.687903
2,ISS,Group2-From-Best-Response-FirstLine,0.994479
3,ISS,Group3-From-Best-Response-FirstLine,0.038867
4,ISS,Group4-From-Best-Response-FirstLine,0.100539


# Translocation [Del + Gain]

In [17]:
clinical = clinical_data[['del13q14', 'del13q34', 'del17p13', 'gain1q21']  + var_resp].copy()

for col in clinical.columns:
    if clinical[col].dtype == 'object':
        clinical[col] = pd.factorize(clinical[col])[0] + 1
        
clinical[clinical.isnull()] = 0

#for c in ['Best-Response-FirstLine', 'Patient-FirstResponse', 'First-line-Therapy-Class', 'First-line-Starting-Treatment-Regimen']:
#    del clinical[c]

clinical.head()

,del13q14,del13q34,del17p13,gain1q21,Days-to-Progression,Greater18Months-Days-to-Progression,Group1-From-Best-Response-FirstLine,Group2-From-Best-Response-FirstLine,Group3-From-Best-Response-FirstLine,Group4-From-Best-Response-FirstLine,Group5-From-Best-Response-FirstLine,Best-Response-FirstLine-ID
MMRF,,,,,,,,,,,,
1007,0,0,0,0,326.0,0,1,1,1,1,0,1
1011,0,0,0,0,1612.0,1,1,1,0,0,0,2
1013,0,0,0,0,1638.0,1,1,0,0,0,0,3
1016,1,1,1,1,691.0,1,1,1,0,0,0,2
1017,0,0,0,0,507.0,0,1,1,0,0,0,2


In [18]:
pearsons = compute_pearson(clinical.copy(), var_resp)

pearsons.to_csv('data/output/correlation_delgain_x_response_vars_pearson.csv', sep=',', index=False)

pearsons.head()

32


,IND_VAR,DEP_VAR,pearson
0,del13q14,Days-to-Progression,-0.024963
1,del13q14,Greater18Months-Days-to-Progression,-0.018991
2,del13q14,Group1-From-Best-Response-FirstLine,-0.061528
3,del13q14,Group2-From-Best-Response-FirstLine,-0.007486
4,del13q14,Group3-From-Best-Response-FirstLine,0.046294


In [19]:
distcorrs = compute_distcorr(clinical.copy(), var_resp)

distcorrs.to_csv('data/output/correlation_delgain_x_response_vars_distcorr.csv', sep=',', index=False)

distcorrs.head()

32


,IND_VAR,DEP_VAR,distcorr
0,del13q14,Days-to-Progression,1.024963
1,del13q14,Greater18Months-Days-to-Progression,1.018991
2,del13q14,Group1-From-Best-Response-FirstLine,1.061528
3,del13q14,Group2-From-Best-Response-FirstLine,1.007486
4,del13q14,Group3-From-Best-Response-FirstLine,0.953706


In [20]:
mics = compute_mic(clinical.copy(), var_resp)

mics.to_csv('data/output/correlation_delgain_x_response_vars_mic.csv', sep=',', index=False)

mics.head()

32


,IND_VAR,DEP_VAR,mic
0,del13q14,Days-to-Progression,0.183779
1,del13q14,Greater18Months-Days-to-Progression,0.000501
2,del13q14,Group1-From-Best-Response-FirstLine,0.004385
3,del13q14,Group2-From-Best-Response-FirstLine,0.001212
4,del13q14,Group3-From-Best-Response-FirstLine,0.004255


In [21]:
kss = compute_ks(clinical.copy(), var_resp)

kss.to_csv('data/output/correlation_delgain_x_response_vars_ks.csv', sep=',', index=False)

kss.head()

32


,IND_VAR,DEP_VAR,ks
0,del13q14,Greater18Months-Days-to-Progression,1.000000
1,del13q14,Group1-From-Best-Response-FirstLine,0.603568
2,del13q14,Group2-From-Best-Response-FirstLine,0.999997
3,del13q14,Group3-From-Best-Response-FirstLine,0.664689
4,del13q14,Group4-From-Best-Response-FirstLine,1.000000


# Translocation [t]

In [22]:
clinical = clinical_data[['t11-14_CCND1', 't12-14_CCND2', 't14-16_MAF', 't14-20_MAFB', 't4-14_WHSC1', 't6-14_CCND3', 't8-14_MAFA', 't8-14_MYC']  + var_resp].copy()

for col in clinical.columns:
    if clinical[col].dtype == 'object':
        clinical[col] = pd.factorize(clinical[col])[0] + 1
        
clinical.columns = [col.lower() if '_' in col else col for col in clinical.columns]

clinical[clinical.isnull()] = 0

#for c in ['Best-Response-FirstLine', 'Patient-FirstResponse', 'First-line-Therapy-Class', 'First-line-Starting-Treatment-Regimen']:
#    del clinical[c]

clinical.head()

,t11-14_ccnd1,t12-14_ccnd2,t14-16_maf,t14-20_mafb,t4-14_whsc1,t6-14_ccnd3,t8-14_mafa,t8-14_myc,Days-to-Progression,Greater18Months-Days-to-Progression,Group1-From-Best-Response-FirstLine,Group2-From-Best-Response-FirstLine,Group3-From-Best-Response-FirstLine,Group4-From-Best-Response-FirstLine,Group5-From-Best-Response-FirstLine,Best-Response-FirstLine-ID
MMRF,,,,,,,,,,,,,,,,
1007,0,0,0,0,0,0,0,0,326.0,0,1,1,1,1,0,1
1011,0,0,0,0,0,0,0,0,1612.0,1,1,1,0,0,0,2
1013,0,0,0,0,0,0,0,0,1638.0,1,1,0,0,0,0,3
1016,1,1,1,1,1,1,1,1,691.0,1,1,1,0,0,0,2
1017,0,0,0,0,0,0,0,0,507.0,0,1,1,0,0,0,2


In [23]:
pearsons = compute_pearson(clinical.copy(), var_resp)

pearsons.to_csv('data/output/correlation_t_x_response_vars_pearson.csv', sep=',', index=False)

pearsons.head()

64


,IND_VAR,DEP_VAR,pearson
0,t11-14_ccnd1,Days-to-Progression,-0.077520
1,t11-14_ccnd1,Greater18Months-Days-to-Progression,-0.059917
2,t11-14_ccnd1,Group1-From-Best-Response-FirstLine,-0.092580
3,t11-14_ccnd1,Group2-From-Best-Response-FirstLine,-0.041135
4,t11-14_ccnd1,Group3-From-Best-Response-FirstLine,0.039982


In [24]:
distcorrs = compute_distcorr(clinical.copy(), var_resp)

distcorrs.to_csv('data/output/correlation_t_x_response_vars_distcorr.csv', sep=',', index=False)

distcorrs.head()

64


,IND_VAR,DEP_VAR,distcorr
0,t11-14_ccnd1,Days-to-Progression,1.077520
1,t11-14_ccnd1,Greater18Months-Days-to-Progression,1.059917
2,t11-14_ccnd1,Group1-From-Best-Response-FirstLine,1.092580
3,t11-14_ccnd1,Group2-From-Best-Response-FirstLine,1.041135
4,t11-14_ccnd1,Group3-From-Best-Response-FirstLine,0.960018


In [25]:
mics = compute_mic(clinical.copy(), var_resp)

mics.to_csv('data/output/correlation_t_x_response_vars_mic.csv', sep=',', index=False)

mics.head()

64


,IND_VAR,DEP_VAR,mic
0,t11-14_ccnd1,Days-to-Progression,0.169614
1,t11-14_ccnd1,Greater18Months-Days-to-Progression,0.003062
2,t11-14_ccnd1,Group1-From-Best-Response-FirstLine,0.006264
3,t11-14_ccnd1,Group2-From-Best-Response-FirstLine,0.001930
4,t11-14_ccnd1,Group3-From-Best-Response-FirstLine,0.004421


In [26]:
kss = compute_ks(clinical.copy(), var_resp)

kss.to_csv('data/output/correlation_t_x_response_vars_ks.csv', sep=',', index=False)

kss.head()

64


,IND_VAR,DEP_VAR,ks
0,t11-14_ccnd1,Greater18Months-Days-to-Progression,0.990284
1,t11-14_ccnd1,Group1-From-Best-Response-FirstLine,0.568847
2,t11-14_ccnd1,Group2-From-Best-Response-FirstLine,0.997521
3,t11-14_ccnd1,Group3-From-Best-Response-FirstLine,0.913496
4,t11-14_ccnd1,Group4-From-Best-Response-FirstLine,0.999944
